In [125]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.io as pio
import re
pio.renderers.default = "vscode"
# pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format


In [163]:
def extract_volume(product_name):
    volume_regex = r'(\d+(\.\d+)?\s*(ml|l))'  # Regex pattern to match both 'ml' and 'l' formats
    #match = re.search(volume_regex, product_name)
    match = re.search(volume_regex, str(product_name))  # Ensure product_name is converted to string

    if match:
        return match.group(1)
    else:
        return 'unspecified'
    
def clean_product_name(product_name):
    # volume_regex = r'\b\d+(\.\d+)?\s*(ml|l)\b'  # '\b' for word boundaries
    # pattern = r'\b\d+(\.\d+)?\s*(ml|l)\b|bli - |gof - '  # '\b' for word boundaries
    pattern = r'\b\d+(\.\d+)?\s*(ml|l)\b|bli - |gof - |- resell ecer| - resell dus|-resell dus|- dus|cabang - '  # '\b' for word boundaries

    return re.sub(pattern, '', str(product_name))  # Ensure product_name is converted to string


In [136]:
df = pd.read_csv("full_iseller_data.csv")

# get datetime columns
df['order_date'] = pd.to_datetime(df['order_date'])
# df['hour'] = df['order_date'].dt.hour
# df['day'] = df['order_date'].dt.day
# df['month'] = df['order_date'].dt.month
# df['week'] = df['order_date'].dt.isocalendar().week

# get location columns, get coordinates ?
df['location'] = df['outlet_name'].str.split(' - ').str[0]

# standardize naming
df['product_name'] = df['product_name'].str.lower()
df['Volume'] = df['product_name'].apply(extract_volume)


C:\Users\ShaanBarca\AppData\Local\Temp\ipykernel_25460\2899326464.py:1: DtypeWarning:

Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.



In [164]:
df['clean_product_name'] = df['product_name'].apply(clean_product_name)
df['clean_product_name'] = df['clean_product_name'].str.strip()

In [97]:
df = df[df['location'].isin(['Bekasi', 'Jaksel', 'Cibubur', 'Jakut', 'Tangerang', 'Bandung' ])]

In [98]:
def filter_dataframe(df, date_start=None, date_end=None, product_category=None, product_name=None):
    filtered_df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    
    # Apply filters based on the provided parameters
    if date_start is not None:
        filtered_df = filtered_df[filtered_df['order_date'] >= pd.to_datetime(date_start)]
    if date_end is not None:
        filtered_df = filtered_df[filtered_df['order_date'] <= pd.to_datetime(date_end)]
    if product_category is not None:
        filtered_df = filtered_df[filtered_df['product_category'] == product_category]
    if product_name is not None:
        filtered_df = filtered_df[filtered_df['product_name'] == product_name]
    
    return filtered_df



product_quantity = df.groupby('product_name')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=False)

In [99]:
def plot_barchart(df, num_products, title):
    df = df.head(num_products)
    fig = px.bar(df, x='product_name', y='quantity', title=title, color='product_name',
                 labels={
                 "product_name": "Product Name",
                 "quantity": "Quantity",
              
             })
  
    return fig.show()

    

In [100]:
def get_total_product_sales(df, date_start=None, date_end =None, product_category=None, product_name=None):
    product_quantity = df.groupby('product_name')['quantity'].sum().reset_index()
    return product_quantity

In [101]:
plot_barchart(product_quantity, 20, title="Top Selling Items")

In [102]:
def plot_line_chart(df, group_by="hour"):
    agg_df = df.groupby(group_by)['quantity'].sum().reset_index()
    fig = px.line(agg_df, x=group_by, y='quantity')
    return fig.show()

In [103]:
plot_line_chart(df, "month")

In [104]:
# def plot_piechart
fig = px.pie(product_quantity.head(10), values='quantity', names='product_name')
fig.show()


In [105]:
df['quantity'].sum()


159278.0

In [106]:
df['location'].value_counts()


location
Bekasi       37499
Jaksel       10921
Cibubur       8454
Jakut         6846
Tangerang     4574
Bandung       1311
Name: count, dtype: int64

In [107]:
df.groupby('product_type')['quantity'].sum().reset_index().sort_values(by="quantity", ascending=False).head()

,product_type,quantity
1,BEER,49269.00
0,ANGGUR,31729.00
14,SOJU,15760.00
11,REGULAR,14974.00
20,WINE,11788.00


In [108]:
df.groupby('product_type')['total_order_amount'].sum().reset_index().sort_values(by="total_order_amount", ascending=False).head()

,product_type,total_order_amount
19,WHISKY,3405332213.00
0,ANGGUR,2130233758.00
1,BEER,1486544578.00
18,VODKA,1463793486.00
7,LIQUOR,1426218037.00


In [198]:
# Product, Category, Revenue, Current Q V Last Q, Unit Sold, Unit Sold THis Q Vs Last Month

current_quarter = df["Quarter"].max()
last_quarter = current_quarter - 1

total_revenue_df = df.groupby(['clean_product_name', 'product_type'])['total_order_amount'].sum().reset_index()
total_revenue_df = total_revenue_df.rename(columns={'total_order_amount': 'total_revenue'})

current_quarter_revenue = df[df['Quarter'] == current_quarter ].groupby(['clean_product_name', 'product_type'])['total_order_amount'].sum().reset_index()
current_quarter_revenue = current_quarter_revenue.rename(columns={'total_order_amount': 'current_quarter_revenue'})

last_quarter_revenue    = df[df['Quarter'] == last_quarter ].groupby(['clean_product_name', 'product_type'])['total_order_amount'].sum().reset_index()
last_quarter_revenue    = last_quarter_revenue.rename(columns={'total_order_amount': 'last_quarter_revenue'})

sales_performance_df = total_revenue_df.merge(current_quarter_revenue, on=["clean_product_name", "product_type"]).merge(last_quarter_revenue, on=["clean_product_name", "product_type"])






In [202]:
df["month"].max()

5

In [201]:
# Product, Category, Revenue, Current Q V Last Q, Unit Sold, Unit Sold THis Q Vs Last Month

sales_performance_df['Quarter_Growth(%)'] = (sales_performance_df['current_quarter_revenue'] - \
                                         sales_performance_df['last_quarter_revenue']) / sales_performance_df['total_revenue']

sales_performance_df.head()

,clean_product_name,product_type,total_revenue,current_quarter_revenue,last_quarter_revenue,Quarter_Growth(%)
0,(es batu) crystalline cup,REGULAR,4851438.00,1442433.00,3409005.00,-0.41
1,a&w root beer kaleng,REGULAR,2094308.00,547516.00,1546792.00,-0.48
2,absolut vodka,VODKA,71879373.00,19134669.00,52744704.00,-0.47
3,alexis anggur hijau,ANGGUR,216934480.00,48557328.00,168377152.00,-0.55
4,alexis anggur merah,ANGGUR,97607059.00,24126968.00,73480091.00,-0.51


In [186]:
current_quarter_revenue.head()

,clean_product_name,product_type,total_order_amount
0,(es batu) crystalline cup,REGULAR,1442433.00
1,a&w root beer kaleng,REGULAR,547516.00
2,absolut vodka,VODKA,19134669.00
3,alexis anggur hijau,ANGGUR,48557328.00
4,alexis anggur merah,ANGGUR,24126968.00


In [110]:
df.groupby(['product_name'])['total_order_amount'].sum().reset_index().head()

,product_name,total_order_amount
0,(es batu) crystalline cup 220ml,3683200.00
1,a&w root beer kaleng 250ml,1302400.00
2,a&w root beer kaleng 250ml - resell ecer,14000.00
3,absolut vodka 750ml,44840000.00
4,absolut vodka 750ml - resell ecer,2355000.00


In [167]:
df['clean_product_name'].nunique()

573

In [175]:
# anggur merah gold cap orang tua anggur merah premium	anggur kolesom 17.5% cap orang tua	
product_count = df['clean_product_name'].value_counts().reset_index().sort_values(by="clean_product_name")


In [147]:
#df[['clean_product_name', 'Volume']].head(200)